In [1]:
%load_ext autoreload
%autoreload 2


import os
os.environ['OMP_NUM_THREADS'] = '1'
import numpy as np  

# LOAD DATA

download data from s3

In [1]:
import pickle
import numpy as np
directory = "/home/mila/r/rebecca.salganik/"
all_data = pickle.load(open(directory + 'ns_music_all_data_ming_20211124','rb'))
all_features_cpu = pickle.load(open(directory + 'tracks_embedding_fullmodel_size64.p','rb'))

In [2]:
df_playlist = all_data['df_playlist']

In [3]:

val_users = all_data['val_user_ids']
train_users = all_data['train_user_ids']
df_playlist_val   = df_playlist[df_playlist['pid'].isin(val_users)]

# R-Precision

In [9]:
R =500

In [ ]:
result = []
for idx, pid in enumerate(val_users):
    
    # get all track ids of the playlist  (SORTED BY POSITION !)
    tracks = df_playlist_val[df_playlist_val['pid']==pid]
    tracks = tracks.sort_values(by='pos')
    all_tids = list(tracks['tid'])
    
    # keep the top MIN[10, len(tracks)/2 ] , predict the rest 
    known_num = min(10, len(all_tids) // 2 )   
    
    # compute average embedding to represent the playlist 
    emb = np.mean(all_features_cpu[all_tids[:known_num]], axis=0)

    # dot product to compute scores of each song, sort 
    data = np.argsort(all_features_cpu.dot(emb))[::-1]    
    
    # compute R-precision 
    predicted_best = data[:R]
    entries = all_tids[known_num:]
    portion = [x for x in entries if x in predicted_best]
    portion = len(portion)/len(entries)   
    result.append([pid, len(all_tids), known_num,  portion])
    
    # 
    if idx % 100 == 0:
        print(idx)
        if idx >  0:
            print(np.mean([x[-1] for x in result]))
            
#pickle.dump( result, open('validataion_rpres_result.p','wb'))

## Refresh Clicks

In [53]:
R = 500
result = []
for idx, pid in enumerate(val_users):
    
    # get all track ids of the playlist  (SORTED BY POSITION !)
    tracks = df_playlist_val[df_playlist_val['pid']==pid]
    tracks = tracks.sort_values(by='pos')
    all_tids = list(tracks['tid'])
    
    # keep the top MIN[10, len(tracks)/2 ] , predict the rest 
    known_num = min(10, len(all_tids) // 2 )   
    
    # compute average embedding to represent the playlist 
    emb = np.mean(all_features_cpu[all_tids[:known_num]], axis=0)

    # dot product to compute scores of each song, sort 
    all_relevant = all_features_cpu.dot(emb)
#     normed_relevant = all_relevant/np.linalg.norm(all_relevant)
#     top_relevance_vals = np.sort(all_relevant)[::-1] [:R]
    indices = np.argsort(all_relevant)[::-1]

    # find overlap 
    predicted_best_indices = indices[:R]
    entries = all_tids[known_num:]
    portion = [x for x in range(len(entries)) if entries[x] in predicted_best_indices]
    
    if len(portion) == 0: 
        clicks = 51 
    else: 
        clicks = portion[0]-1//10

    result.append([pid, len(all_tids), known_num,  clicks])
    
    if idx % 10 == 0:
        print(idx)
        if idx >  0:
            print(np.mean([x[-1] for x in result]))

0
10
7.181818181818182
20
15.761904761904763


KeyboardInterrupt: 

## NDCG

In [43]:
R = 500
result = []
for idx, pid in enumerate(val_users):
    
    # get all track ids of the playlist  (SORTED BY POSITION !)
    tracks = df_playlist_val[df_playlist_val['pid']==pid]
    tracks = tracks.sort_values(by='pos')
    all_tids = list(tracks['tid'])
    
    # keep the top MIN[10, len(tracks)/2 ] , predict the rest 
    known_num = min(10, len(all_tids) // 2 )   
    
    # compute average embedding to represent the playlist 
    emb = np.mean(all_features_cpu[all_tids[:known_num]], axis=0)

    # dot product to compute scores of each song, sort 
    all_relevant = all_features_cpu.dot(emb)
    normed_relevant = all_relevant/np.linalg.norm(all_relevant)
    top_relevance_vals = np.sort(normed_relevant)[::-1] [:R]
    indices = np.argsort(normed_relevant)[::-1]

    # find overlap 
    predicted_best_indices = indices[:R]
    entries = all_tids[known_num:]
    portion = [x for x in entries if x in predicted_best_indices]
    
    
    DCG = top_relevance_vals[0] + np.sum(np.array([top_relevance_vals[i]/np.log2(i) for i in range(2, len(top_relevance_vals))]))

    IDCG = 1 + np.sum(np.array([1/np.log2(i) for i in range(2,len(portion))]))
    
    NDCG = DCG/IDCG

    result.append([pid, len(all_tids), known_num,  NDCG])
    
    if idx % 10 == 0:
        print(idx)
        if idx >  0:
            print(np.mean([x[-1] for x in result]))

0
10
0.03746878980317834
20
0.03421965873758824
30
0.03080625692826479
40
0.031176445745349127
50
0.0290667592411718
60
0.028140710949403486


KeyboardInterrupt: 

In [19]:
print(np.sort(np.array([10,2,3,5,6]))[::-1])

[10  6  5  3  2]


In [39]:
emb = np.mean(all_features_cpu[all_tids[:known_num]], axis=0)
all_relevant = all_features_cpu.dot(emb)
normed_relevant = all_relevant/np.linalg.norm(all_relevant)
top_relevance_vals = np.sort(normed_relevant)[::-1] [:R]
np.max(top_relevance_vals[0])

0.0007914771

In [19]:
overlap_values = all_relevant[portion]
print(len(overlap_values))
print(overlap_values[0])
print(len(portion))

4
50.016388
4


In [40]:
top = np.sum(np.array([np.log2(i) for i in range(2, len(top_relevance_vals))]))
bottom = np.sum(top_relevance_vals[2:])
DCG = top/bottom

DCG2 = top_relevance_vals[0] + np.sum(np.array([top_relevance_vals[i]/np.log2(i) for i in range(2, len(top_relevance_vals))]))

IDCG = 1 + np.sum(np.array([1/np.log2(i) for i in range(2,len(portion))]))

print(DCG, DCG2, IDCG)

print(DCG/IDCG, DCG2/IDCG)

9877.391590957583 0.05467726225629184 2.6309297535714578
3754.3349751353617 0.020782486564708946
